# Open Street Map - Data Wrangling with SQL


## Project Overview

To choose any area of the world in https://www.openstreetmap.org and use data wrangling techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for that part of the world. Finally, use SQL as the data schema to complete your project by storing, querying and aggregating the data.


<img src="manchester_area.jpg">

Area Sampled: 

* Manchester and surrounding areas
* MinLat="53.237" MinLon="-2.588" 
* MaxLat="53.672" MaxLon="-1.877"

Source: https://mapzen.com/data/metro-extracts/metro/manchester_england/

***
## Data Audit

Openstreet map consists of user submitted data, therefore is prone to errors. Before importing into SQL the data will be audited for accuracy and consistency, with the main focus on postal codes and street addresses.
The main issues I noticed when investigating the data were as follows:

* Abbreviated street names (Av for Avenue)
* Incorrect and inconsistent postal codes (format)
* Postal codes a large distance away from government data.


### Tags

 ``mapparsey.py``

Counts of each of the tags in the OSM data:
```python
{'bounds': 1,
 'member': 31592,
 'nd': 2039153,
 'node': 1647945,
 'osm': 1,
 'relation': 2479,
 'tag': 899132,
 'way': 231099}
 ```
 ### Tag type count ###
 ``tags.py``
 
 Tags in OSM are represented as: ``<tag k="addr:country" v="US"/>``
 
 We wish to transform these into a dictionary value such as this: {"address": {"country": "US"}}
 
 ``tags.py`` is used to display a list of keys found in the file with the following defitions:
 
  - **"lower",** for tags that contain only lowercase letters and are valid
  - **"lower_colon"**, for otherwise valid tags with a colon in their names
  - **"problemchars"**, for tags with problematic characters
  - **"other"**, for other tags that do not fall into the other three categories.
  
#### Output:
  
 ```python
 {'lower': 670084, 'lower_colon': 106435, 'other': 122610, 'problemchars': 3}
 ```
 
#### Problem Characters consisted of the set:
 
```python
set(['floor_area:level -1', 'recycling:white goods', 'baby change'])
```
 
Which could be resolved by replace white space from characters with an underscore
 

The ``others`` set consisted of a many different name patterns. Most entries had one or more letters capitalized, which was easily fixed using the str.lower() method.

### Auditing street names
``audit_streets.py``

The data didn't suggest a large problem with street names, there were quite a few abbrievation issues that I didn't think of prior to the investigation, so fixed them during parsing. Examples include ln -> Lane, sq -> Square.
A number of the street names were not capitalized correctly, this was a simple fix.

A quick scan over the invalid names reveals some obviously invalid street names such as **"Avenuehttps://streaming.media.ccc.de/33c3/"** which have been removed.

***

### Auditing post codes
``audit_postcodes.py``

ref: https://stackoverflow.com/questions/164979/uk-postcode-regex-comprehensive

Post codes were audited using the following regular expression



>#### Explanation:
> ``^([A-Za-z]{1,2}[0-9]{1,2}[A-Za-z]?[ ]?)([0-9]{1}[A-Za-z]{2})$ ``
> * expect 1 or 2 a-z chars, upper or lower fine
> * expect 1 or 2 numbers
> * expect 0 or 1 a-z char, upper or lower fine
> * optional space allowed
> * expect 1 number
> * expect 2 a-z, upper or lower fine
 

Upon auditing the entire data set I found **7047 good post codes** - and **22 badly formatted postcodes** which seems like a fairly ratio of good to bad postcodes.

A sample of the bad codes were as follows:
```python 
['M34', 'M3', 'M4 1', 'M23', 'SK4', 'M1', 'SK16', 'BL5', 'BL9', 'SK5', 'SK5', 'SK5', 'BL5', 'SK8', 'M32', 'M25', 'M9', 'M15', 'M50 3SA;M50 3SB', 'CW9', 'SK5 7', 'M44'] ```

I decided to dig further into the malformed postal codes.  Around ten were missing the tail of the postcode, this could be due to human error when inserting. e.g. Greenbrow Road in the dataset has a postal code of **M23**, a google search reveals the correct postal code is  **M23 2ET**.  Seeing as there were such a small amount of invalid postal codes they were dropped from the dataset.

Next I obtained examined the locality of the postal codes by looking at the first two characters:

```python 
{'sk': 1, 'HX': 19, 'OL': 546, 'Wa': 1, 'WA': 278, 'BL': 246, 'M9': 18, 'M5': 53, 'm2': 1, 'SK': 2775, 'M4': 146, 'M7': 20, 'M6': 17, 'M1': 1799, 'M3': 284, 'M2': 398, 'WN': 34, 'CW': 388, 'M8': 5, 'HD': 18})
```

All of these were in the area in and around Manchester so did not have to be modified.  It is worth noting that lowercase postal codes are transformed to uppercase during the final transformation.

#### Gold Standard Data
To audit postcdoes for accuracy I compared OSM postcode latitude and longituide coordinates to those found in a national dataset provided by the government.  I created a subset of the goverment data then I used the package ``gpxpy`` to compare the distance between the two sets of coordinates. Distance in KM can be worked out as follows:

```python
lat1 = float(valid_osm_postcode[postcode][0]['lat'])
lon1 = float(valid_osm_postcode[postcode][0]['lon'])

lat2 = float(valid_postcode_data[postcode]['lat'])
lon2 = float(valid_postcode_data[postcode]['lon'])

dist = gpxpy.geo.haversine_distance(lat1, lon1, lat2, lon2)
distance[postcode] = dist / 1000
```

##### Results
* Average distance = **0.058 km**
* Minimum distance = **0.001 km** for the postcode **WA3 6AE**
* Maximum distance = **0.953 km** for the postcode **M60 4EP**

An average distance of 58 meters away seems like the data is *fairly* accurate. 
The largest distance postcode is nearly a kilometer away, Although postcodes can cover a large area in the UK - I think it's safe to say that has been added incorrectly.  The following post codes below were removed from the OSM dataset as they were over half a kilometer away from the gold standard dataset.

|POSTCODE | DISTANCE(KM)|
|------------------ |
| M19 2SY |  0.525  |
| M60 4EP |  0.952  |
| M15 6FD |  0.689  |
| SK5 6XD |  0.718  |
| M17 1TD |  0.566  |



### Data Cleaning Summary

Overall it looks like the data for the Manchester area doesn't require too much manipulation.

After auditing the data the following changes need to be made:
* Capitalization of street names
* Capitalization of postal codes
* Remove postcodes with large distance difference to government dataset.
* Conversion of abbreviated street names to proper street names
* Removal of erroneous postal codes
* Removal of erroneous street names

## Data Overview 

Data was transformed into CSV files and then imported into an SQLite3 Database.  During the transformation process I had problems validating the CSV to meet the schema requirements when a node didn't have a user or userID present.  

The following function looks up the default value in the schema for node or way types:

```python
def default_val(nodeval, attrib):

    _type = schema.schema[nodeval]['schema'][attrib].get('coerce', "None")
    if _type == int:
        _type = 0
    
    return _type
```

#### File Sizes
```
manchester_england.osm ......... 350 MB
osm.db ......................... 196 MB
nodes.csv ...................... 127 MB
nodes_tags.csv ................. 10.9 MB
ways.csv ....................... 12.9 MB
ways_nodes.cv .................. 46.1 MB 
ways_tags.csv .................. 19.2 MB
 
```

#### Counts

```SQL
SELECT COUNT(*) FROM nodes;
```
NODES: 1647945

```SQL
SELECT COUNT(*) FROM ways;
```
WAYS: 231099

#### Count of McDonalds
```SQL
Select nodes_tags.value, COUNT(*) as num
FROM nodes_tags
    JOIN (Select distinct(id) FROM nodes_tags WHERE nodes_tags.value = "restaurant" 
    OR nodes_tags.value = "fast_food") as r
ON nodes_tags.id=r.id
WHERE nodes_tags.value like "%mcd%"
```

Result: 32 McDonalds

#### Top Contributing Users

```SQL
SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user
ORDER BY num DESC
LIMIT 10;
```

|Username   | Edits|
|----------------------|
|"alterain" |	"107657"   |
|"Steeley"	|"97765"|
|"poshbakerloo"|	"63824"|
|"RobChafer"|	"47438"|
|"InsertUser"|	"47309"|
|"Chris Parker"|	"46180"|
|"richardwest"|	"44360"|
|"Nigel Greens"|	"43167"|
|"krispyduck"|	"40536"|


#### Most popular restaurants
No surprise here with Manchester and it's "Curry Mile", a long stretch of road full of Indian restaurants.
```SQL
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') as r
    ON nodes_tags.id=r.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
```
|Cuisine|Amount|
|-----------------|
|"indian"|	"68"|
|"italian"|	"52"|
|"chinese"|	"46"|
|"pizza"|	"19"|
|"thai"|	"15"|
|"american"|	"9"|
|"asian"|	"9"|
|"japanese"|	"9"|
|"steak_house"|	"9"|
|"regional"|	"7"|

#### Count of religious tags

```SQL
SELECT tags.value AS "Religion", COUNT(*) AS "Count" 
FROM (SELECT * FROM nodes_tags
      UNION ALL 
      SELECT * FROM ways_tags) tags
WHERE tags.key='religion'
GROUP BY tags.value
ORDER BY "Count" DESC
LIMIT 3;
```
|Religion|Amount
|-----------|---------|
|"christian"|	"691"|
|"muslim"|	"35"|
|"jewish"|	"13"|


## Conclusion

I've demonstrated data wrangling, cleaning and exploration skills with the OSM dataset using Python and SQL.

I noticed that there is a surprisingly small number of postcodes compared to way and node data - after doing some further research I discovered nationally only 3% of postcodes are mapped in OSM.  This figure seems very low to me, perhaps a drive can be made to programically map postal code data found in official sources to  OSM? http://sk53-osm.blogspot.co.uk/2013/12/british-postcodes-on-openstreetmap.html

I'm surprised at the accuracy of the postal code data that was submitted with very few failing to pass the regex test and most falling within a suitable range of the official dataset.

## Other Ideas for Analysis

Looking forward I understand that map data is volitile and changes frequently, to keep up with the pace of change we could consider applying a heat map to the Manchester area, highlighting zones have not been updated recently by looking at the timestamp values associate with each node or way.  By doing this areas that have not been updated recently would be targetted for a review.
On the other hand, a negative to this is that areas may be updated unnecessarily - simply to introduce a new timestamp value which could cause inaccurate entries.

Another possible solution moving forward could be creating a bot that automatically checks postal code coordinates fall within the bounds of the government data. Issues could arrise here as postal codes can cover a large area in the UK, so when do we set the maximum distance? 
These positive and negative impacts should be weighted before implementing this kind of improvement to the process.



## References
- Gov Data -  https://data.gov.uk/dataset/national-statistics-postcode-lookup-uk
- OSM Data - https://mapzen.com/data/metro-extracts/metro/manchester_england/
- Sample Project - https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md
- Regex - https://stackoverflow.com/questions/164979/uk-postcode-regex-comprehensive
- UK Postcode  Status - http://sk53-osm.blogspot.co.uk/2013/12/british-postcodes-on-openstreetmap.html